In [1]:
script("tarballs");

In [2]:
@P3 = unpack_tarball("fano-v3d.tgz");

In [3]:
@P4 = unpack_tarball("fano-v4d.tgz");

In [ ]:
@P5 = unpack_tarball("fano-v5d.tgz");

In [4]:
sub Bilateral {
    $rows = $_[0]->rows();
    $cols = $_[0]->cols();
    $b = 0;
    
    for (my $row = 0; $row < $rows; $row += 1) {
        $neg_orth = 1;
        $basis = 1;
        $summa = 0;
        for (my $col = 0; $col < $cols; $col += 1) {
            $elem = $_[0]->elem($row, $col);
            $summa += $elem;
            if ($elem < 0 or $elem > 1 or $summa > 1) {$basis = 0;}
            if ($elem > 0)  {$neg_orth = 0;}
        }
        if ($basis == 1 and $summa == 0) {$basis = 0;}
        if (not ($basis == 1 or $neg_orth == 1)) {return 0;}
        $b = $b + $basis;
    }
    if ($b != $cols) {return 0;}
    return 1;
}

sub SpecialFacet {
    $rows = $_[0]->rows();
    $cols = $_[0]->cols();
    $b = 0;
    
    for (my $row = 0; $row < $rows; $row += 1) {
        $basis = 1;
        $summa = 0;
        for (my $col = 0; $col < $cols; $col += 1) {
            $elem = $_[0]->elem($row, $col);
            $summa += $elem;
            if ($elem < 0 or $elem > 1 or $summa > 1) {$basis = 0;}
        }
        if ($basis == 1 and $summa == 0) {$basis = 0;}
        if ($summa > 1) {return 0;}
        if ((not $basis) and $summa == 1) {return 0;}
        $b = $b + $basis;
    }
    if ($b != $cols) {return 0;}
    
    for (my $col = 0; $col < $cols; $col += 1) {
        $summa = 0;
        for (my $row = 0; $row < $rows; $row += 1) {
            $elem = $_[0]->elem($row, $col);
            $summa += $elem;
        }
        if ($summa < 0) {return 0;}
    }
    return 1;
}

In [5]:
print "P\t|V|\t|F|\t|S|\t|B|\t|A|\t|X|\t|E|\n";

$pol_number = 0;
foreach my $polytope (@P3) {
#for ($pol_number = 7522; $pol_number < 7622; $pol_number += 1) {
#for ($pol_number = 72156; $pol_number < 72256; $pol_number += 1) {
#    $polytope = load("fano-v6d/F.6D.$pol_number.poly");
#    $polytope = load("fano-v7d-7/fano.pnew.7D.0$pol_number.poly");
    
    $p = polarize($polytope);
    $nfacets = $p->N_FACETS;
    $nvertices = $p->N_VERTICES;
    $vertices = $p->VERTICES;
    $facets = $p->VERTICES_IN_FACETS;
    $incidence_matrix = dense($facets);
    
    #print "Polytope $pol_number\nVertices\n", rows_labeled($vertices), "\n"; # Polytopes/info
    #print "Special Facets: ", $p->SPECIAL_FACETS, "\n\n"; # Polytopes
    
    $Sp = 0;
    $Bil = 0;
    $And = 0;
    $Xor = 0;
    $Eq = $nfacets;
    
    @special = ();
    @bilateral = ();
    
    for (my $facet = 0; $facet < $nfacets; $facet += 1) {
        @f = ();
        for (my $vertex = 0; $vertex < $nvertices; $vertex += 1) {
            if ($incidence_matrix->elem($facet, $vertex)) { push(@f, $vertex); }
        }
        $facet_set = new Set(@f);
        
        $matrix = $vertices->minor($facet_set, ~[0]);
        $inverse_matrix = inv($matrix);
        $transformed = $vertices->minor(All, ~[0]) * $inverse_matrix;
        #print "Facet ", $facets->row($facet), " to basis:\n$transformed", "\nby matrix\n", transpose($inverse_matrix), "\n"; # info
        
        $sp = SpecialFacet($transformed);
        $bil = Bilateral($transformed);
        
        $Sp += $sp;
        $Bil += $bil;
        $And += $sp * $bil;
        $Xor += ($sp + $bil) % 2;
        $Eq -= ($sp + $bil) % 2;
        
        push(@special, $sp);
        push(@bilateral, $bil);
    }
    
    #print "Sp\tBil\tFacet\n"; # Polytopes
    for (my $facet = 0; $facet < $nfacets; $facet += 1) {
        #print "$special[$facet]\t$bilateral[$facet]\t", $facets->row($facet), "\n"; # Polytopes
    }
    print "$pol_number\t$nvertices\t$nfacets\t$Sp\t$Bil\t$And\t$Xor\t$Eq\n"; #conc
    #print "\n\n"; # Polytopes/info
    
    $pol_number += 1;
}

0	6	8	2	1	0	3	5
1	5	6	3	3	0	6	0
2	7	10	4	2	0	6	4
3	8	12	4	0	0	4	8
4	7	10	2	2	0	4	6
5	6	8	4	4	0	8	0
6	6	8	2	2	0	4	4
7	7	10	4	0	0	4	6
8	7	10	4	2	0	6	4
9	8	12	12	0	0	12	0
10	6	8	4	1	0	5	3
11	6	8	4	4	0	8	0
12	6	8	8	0	0	8	0
13	5	6	4	2	0	6	0
14	5	6	3	3	0	6	0
15	6	8	8	8	8	0	8
16	5	6	6	6	6	0	6
17	4	4	4	4	4	0	4

In [6]:
$p = polarize(load("fano-v3d/F.3D.0000.poly"));
$nfacets = $p->N_FACETS;
$nvertices = $p->N_VERTICES;
$vertices = $p->VERTICES;
$facets = $p->VERTICES_IN_FACETS;
$incidence_matrix = dense($facets);

@special = ();
@bilateral = ();

for (my $facet = 0; $facet < $nfacets; $facet += 1) {
    @f = ();
    for (my $vertex = 0; $vertex < $nvertices; $vertex += 1) {
        if ($incidence_matrix->elem($facet, $vertex)) { push(@f, $vertex); }
    }
    $facet_set = new Set(@f);
    
    $transformed = $vertices->minor(All, ~[0]) * inv($vertices->minor($facet_set, ~[0]));
    
    push(@special, SpecialFacet($transformed));
    push(@bilateral, Bilateral($transformed));
}
print @special, "\n";
print @bilateral, "\n";

11000000
00000001